In [1]:
import pandas as pd
import numpy as np
from flask import render_template, request, Flask
from sklearn.preprocessing import RobustScaler
import pickle

def preprocess(data):
    df = pd.DataFrame(data)
    ######################################################################
    df.drop('Booking_ID',axis=1,inplace=True)

    df['date of reservation'] = df['date of reservation'].replace('2018-2-29','3/1/2018')
    df['month']=(df['date of reservation'].str.split('/',expand=True)[0]).astype(int)
    df['day']=df['date of reservation'].str.split('/',expand=True)[1].astype(int)
    df['year']=df['date of reservation'].str.split('/',expand=True)[2].astype(int)
    df.drop('date of reservation',axis=1,inplace=True)
    
    col='number of adults'
    replace_dict={0:'other',1:'1',2:'2',3:'3',4:'other',5:'other',6:'other'}
    df[col] = df[col].replace(replace_dict)
    
    col='number of children'
    replace_dict={0:'0',1:'1',2:'2',3:'other',4:'other',5:'other',6:'other',7:'other',8:'other',9:'other'
                  ,10:'other',11:'other',12:'other'}
    df[col] = df[col].replace(replace_dict)
    
    col='number of weekend nights'
    replace_dict={0:'0',1:'1',2:'2',3:'other',4:'other',5:'other',6:'other',7:'other',8:'other',9:'other'
                  ,10:'other',11:'other',12:'other'}
    df[col] = df[col].replace(replace_dict)
    
    col='number of week nights'
    replace_dict={0:'0',1:'1',2:'2',3:'3',4:'4',5:'5',6:'other',7:'other',8:'other',9:'other'
                  ,10:'other',11:'other',12:'other',13:'other',14:'other',15:'other'
                 ,16:'other',17:'other',18:'other'}
    df[col] = df[col].replace(replace_dict)
    
    col='room type'
    replace_dict={'Room_Type 1':'1'  ,'Room_Type 2':'other','Room_Type 3':'other','Room_Type 4':'4','Room_Type 5':'other'
                 ,'Room_Type 6':'other','Room_Type 7':'other'}
    
    df[col] = df[col].replace(replace_dict)
    
    col='market segment type'
    replace_dict={'Corporate':'other'
                 ,'Aviation':'other','Complementary':'other'}
    
    df[col] = df[col].replace(replace_dict)

    col='P-C'
    replace_dict={0:'NO',1:'YES',2:'YES',3:'YES',4:'YES',5:'YES',6:'YES',11:'YES',13:'YES'}
    df[col] = df[col].replace(replace_dict)
    
    col='special requests'
    replace_dict={0:'0'  ,1:'1',  2:'2',  3:'other', 4:'other',5:'other'}
    df[col] = df[col].replace(replace_dict)
    
    col='P-not-C'
    replace_dict={0:'YES'  ,5:'NO',  3:'NO',  1:'NO', 12:'NO', 19:'NO',  2:'NO', 15:'NO', 17:'NO',  4:'NO', 13:'NO',  6:'NO'
                  , 14:'NO', 34:'NO', 18:'NO', 16:'NO',  8:'NO',10:'NO', 23:'NO', 11:'NO', 49:'NO', 47:'NO', 53:'NO',  7:'NO',
                  9:'NO', 33:'NO', 22:'NO', 24:'NO', 52:'NO', 21:'NO', 48:'NO', 28:'NO', 39:'NO', 20:'NO',38:'NO', 26:'NO',
                  51:'NO', 42:'NO', 37:'NO', 35:'NO', 56:'NO', 44:'NO', 27:'NO', 55:'NO', 45:'NO', 30:'NO', 57:'NO', 31:'NO'
                  , 54:'NO', 43:'NO', 58:'NO',41:'NO', 29:'NO', 40:'NO', 36:'NO', 32:'NO',50:'NO', 25:'NO', 46:'NO' }
    df[col] = df[col].replace(replace_dict)
    #################################################################################################################
    hot=['number of adults', 'number of children', 'number of weekend nights',
       'number of week nights', 'type of meal', 'car parking space',
       'room type', 'market segment type', 'repeated', 'P-C',
       'P-not-C', 'special requests'
         ]
    df=pd.get_dummies(columns=hot,data=df)
    ###################################################################################################################
    before_scaling=pd.read_csv("after_encoding_.csv").drop('Unnamed: 0',axis=1)
    
    bfscols = before_scaling.columns

    new_df = pd.DataFrame(columns=bfscols)
    new_df = new_df.append(before_scaling.iloc[0]).astype(before_scaling.dtypes.to_dict(), copy=False)
    
    for col in new_df.select_dtypes(include='bool').columns:
        new_df[col] = False
    for col in new_df.select_dtypes(np.number).columns:
        new_df[col] = 0
        
        
    for col in df.columns:
        new_df.loc[0,col]=df.loc[0,col]

    rr=RobustScaler()
    rr.fit(before_scaling)
    scaled=rr.transform(new_df)
    
    
    return scaled
    
    
    
    


In [ ]:


app = Flask(__name__)

# Load the pre-trained model
model = pickle.load(open("random_forest_model_pickle.pkl", "rb"))

# Create a scaler (you might need to adjust this based on how you scaled the data during training)
robust_scaler = RobustScaler()

@app.route("/")
def home():
    return render_template("index.html", prediction_text="")

@app.route("/predict", methods=["POST"])
def predict():
    try:
        
        
        # Extract features from form data
        Booking_ID = request.form['Booking_ID']
        number_of_adults = int(request.form['Number of Adults'])
        number_of_children = int(request.form['number of children'])
        number_of_weekend_nights = int(request.form['number of weekend nights'])
        number_of_week_nights = int(request.form['number of week nights'])
        type_of_meal = request.form['type of meal']
        car_parking_space = int(request.form['car parking space'])
        room_type = request.form['room type']
        lead_time = int(request.form['lead time'])
        market_segment_type = request.form['market segment type']
        repeated = int(request.form['repeated'])
        P_C = int(request.form['P-C'])
        P_not_C = int(request.form['P-not-C'])
        average_price = float(request.form['average price'])
        special_requests = int(request.form['special requests'])
        date_of_reservation = request.form['date of reservation']

        # Create a DataFrame from the form data
        data = {
            'Booking_ID': [Booking_ID],
            'number of adults': [number_of_adults],
            'number of children': [number_of_children],
            'number of weekend nights': [number_of_weekend_nights],
            'number of week nights': [number_of_week_nights],
            'type of meal': [type_of_meal],
            'car parking space': [car_parking_space],
            'room type': [room_type],
            'lead time': [lead_time],
            'market segment type': [market_segment_type],
            'repeated': [repeated],
            'P-C': [P_C],
            'P-not-C': [P_not_C],
            'average price ': [average_price],
            'special requests': [special_requests],
            'date of reservation': [date_of_reservation]
        }
        
        scaled=preprocess(data)
       # Make prediction
        prediction = model.predict(scaled)
        if prediction==0:
            m='CANCELED'
        else:
            m='NOT_CANCELED'

        # Display the prediction on the web page
        return render_template("index.html", prediction_text=f"The predicted booking status is {m}")

    except Exception as e:
        return render_template("index.html", prediction_text=f"Error: {str(e)}")


if __name__ == '__main__':
    app.run(debug=False)
   


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Nov/2023 18:33:44] "GET / HTTP/1.1" 200 -
C:\Users\agoud\AppData\Local\Temp\ipykernel_11832\511848329.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(before_scaling.iloc[0]).astype(before_scaling.dtypes.to_dict(), copy=False)
127.0.0.1 - - [24/Nov/2023 18:33:49] "POST /predict HTTP/1.1" 200 -
